# Setup

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import lasio
import pandas as pd
import scipy.signal as signal
import os
import random
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.spatial import distance
from sklearn import preprocessing
from itertools import combinations
import copy
import pywt
import scipy

from lib.well_correlation.dtw.multiple_well_correlation import Dtw, MultipleDtw
from lib.well_correlation.dtw.plotting import plot_cross_section

In [ ]:
%matplotlib inline

plt.rcParams['figure.figsize'] = (18, 10)
plt.rcParams['axes.titlesize'] = 16

In [ ]:
def las2csv(path):
    csv_folder = os.path.join(path, 'CSV')
    
    if not os.path.exists(csv_folder):
        os.makedirs(csv_folder)

    for file in os.listdir(path):        
        if not (file.endswith('.las')):
            continue
    
        las_path = os.path.join(path,file)
        csv_name = file.split('.')[0] + '.csv'
        df_data = lasio.read(las_path).df()
        
        save_path = os.path.join(csv_folder, csv_name)
        df_data.to_csv(save_path)

In [ ]:
def plot_logs(l: list):
    N = len(l)
    
    fig, ax = plt.subplots(figsize=(2 * N, 10))
    
    xmin = np.inf
    xmax = -np.inf
    
    acc = 0
    for i in range(N):
        ax.plot(l[i][:, 1] + acc, l[i][:, 0])
        xmin = min(xmin, np.min(l[i][:, 0]))
        xmax = max(xmax, np.max(l[i][:, 0]))
        acc += np.max(l[i][:, 1])
         
    ax.set(
        ylim = (xmax, xmin),
        ylabel = "Depth",
        xlabel = "Logs"
    )
    
    return fig, ax

# Correlação em Salta

In [ ]:
df = {}
wells = range(1, 7)

for i in wells:
    # dropna: remove valores nan
    df[i] = pd.read_csv(f'data/SaltaCSV/well_{i}_GR.csv').sort_values('DEPT').dropna()

In [ ]:
mdtw = MultipleDtw(norm=0.125)

logs = []
for i in wells:
    logs.append(mdtw.add_log(df[i][['DEPT', 'GR']].dropna().values))

for i, j in combinations(logs, 2):
    mdtw.add_correlation(i, j)

warped = mdtw.delta_query()
plot_logs(list(map(lambda x: x.values, warped)))

# Onduletas 🌊

In [ ]:
# def wavelet(data, onduleta = signal.ricker, width = (32,768)):
#     widths = np.arange(width[0], width[1])
#     cwt = signal.cwt(data, onduleta, widths).T
#     cwt /= np.max(np.abs(cwt))
#     # cwt = (cwt - cwt.mean()) / cwt.std()
#     return cwt

# fig,ax = plt.subplots()
# onduleta = wavelet(df[1]['GR'])
# ax.imshow(onduleta, cmap="seismic", aspect='auto', vmax=0.1, vmin=-0.1)

def wavelet(y, x, scales=32, wavelet = 'morl'):
    coefs, freqs = pywt.cwt(
        data=y,
        scales=range(1, scales),
        wavelet=wavelet,
        sampling_period=x.iloc[0]-x.iloc[1] # iloc: linha 0 pode não existir
    )
    return coefs

def slice(coefs, b):
    return [np.sum(coefs[:b, i]) for i in range(len(coefs[0]))]

def plot_wavelet(y, x, coefs, b, proj, title):
    fig, ax = plt.subplots(nrows=1, ncols=3)

    x_max = np.max(x)
    x_min = np.min(x)
    
    # sinal original
    ax[0].plot(y, x)
    ax[0].set_ylim(x_min, x_max)
    ax[0].set_xlabel('GR')
    
    # coeficientes da wavelet
    coefs_rotated = scipy.ndimage.rotate(coefs, 90)
    ax[1].imshow(coefs_rotated, aspect='auto')
    ax[1].set_xlabel('Coefficients')

    # slice
    ax[2].set_ylim(x_min, x_max)    
    ax[2].barh(
        np.arange(len(proj)),
        proj,
        antialiased=True
    )

    # linha vertical nos coeficientes
    ax[1].plot(
        [b for i in range(len(coefs[0]))],
        range(len(coefs[0])),
        'r-',
        linewidth=1,
        antialiased=True
    )

    # configurações gerais de plotting
    for i in range(len(ax)):
        ax[i].invert_yaxis()
        if i > 0:
            ax[i].get_yaxis().set_visible(False)
        if i > 1:
            ax[i].set_ylim(
                ax[i].patches[0].get_y(),
                ax[i].patches[-1].get_y()
            )
    
    fig = plt.gcf()
    fig.suptitle(f"Well {title}", fontsize=14)

In [ ]:
b = 15
for well in wells:
    x = df[well]['DEPT']
    y = df[well]['GR']
    
    coefs = wavelet(y, x)
    proj = slice(coefs, b)
    plot_wavelet(y, x, coefs, b, proj, well)

# Correlação com Onduletas

In [ ]:
scales = 32
slices = [1]

wavelets = {}

for well in wells:
    x = df[well]['DEPT']
    y = df[well]['GR']
    coef = wavelet(y, x)
    
    wavelets[well] = pd.DataFrame({'DEPT': x})
    for b in slices:
        proj = slice(coef, b)
        wavelets[well][b] = proj

mdtw = MultipleDtw()
logs = []

for well in wells:
    logs.append(mdtw.add_log(wavelets[well].dropna().values))

for i, j in combinations(logs, 2):
    mdtw.add_correlation(i, j)
    
# warped = mdtw.delta_query()
# plot_logs(list(map(lambda x: x.values, warped)))